## Разметим датасет lenta-ru из ДЗ1-2 с помощью deeppavlov

## Импорт библиотек

Импортируем необходимые библиотеки для работы с данными, разметки текстов и загрузки предобученной NER-модели. Фиксируем `random_state` для воспроизводимости результатов.


In [1]:
import pandas as pd
from tqdm import tqdm
from corus import load_lenta
from deeppavlov import build_model
from transformers import AutoTokenizer

random_state = 777

/home/bitcoin/ITMO-DL-NLP/.venv3.10-deeppavlov/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Загрузка датасета

Скачиваем архив с новостями Lenta.ru для дальнейшей обработки и разметки.


In [2]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

--2025-09-25 19:34:35--  https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/87156914/0b363e00-0126-11e9-9e3c-e8c235463bd6?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-09-25T17%3A31%3A10Z&rscd=attachment%3B+filename%3Dlenta-ru-news.csv.gz&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-09-25T16%3A31%3A06Z&ske=2025-09-25T17%3A31%3A10Z&sks=b&skv=2018-11-09&sig=7au0aKCFjRINUeB1yPeQiOJytX5%2BXbcdEVEa469BnAM%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1ODgxODM3NSwibmJmIjoxNzU4ODE4MDc1LCJwYXRoIjoicmVsZWFzZWFzc2V0

## Инициализация NER-модели

Загружаем предобученную модель распознавания именованных сущностей из библиотеки DeepPavlov. Модель основана на BERT и обучена на русскоязычном корпусе Collection3.


In [ ]:
ner_model = build_model('ner_collection3_bert', download=True, install=False)

2025-09-26 13:50:54.425 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/v1/ner/ner_rus_bert_coll3_torch.tar.gz download because of matching hashes
/home/bitcoin/ITMO-DL-NLP/.venv3.10-deeppavlov/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializi

## Подготовка данных

Загружаем датасет новостей Lenta.ru, формируем случайную выборку из 10000 текстов и оставляем только столбец с текстом для дальнейшей обработки.


In [3]:
records = load_lenta('lenta-ru-news.csv.gz')
data = pd.DataFrame(records)
data.columns = ['url', 'title', 'text', 'topic', 'tags', 'date']
data = data.sample(n=10000, random_state=random_state)
data = data[['text']].reset_index(drop=True)
data.head()

,text
0,Американские фондовые рынки открылись 10 авгус...
1,Венесуэла намерена ввести въездные визы для гр...
2,"Ученые из Австралии и Японии, используя новую ..."
3,Авианосец нового поколения для военно-морского...
4,"Тактика сеяния хаоса, которую ведет террористи..."


## Тестирование модели

Проверяем работу NER-модели на одном примере текста, чтобы понять формат выходных данных.


In [4]:
print(ner_model([data["text"][0]]))

[[['Американские', 'фондовые', 'рынки', 'открылись', '10', 'августа', 'резким', 'снижением', 'котировок.', 'За', 'первые', 'минуты', 'торгов', 'индекс', 'Dow', 'Jones', 'упал', 'на', '2,67', 'процента', 'и', 'снова', 'торгуется', 'ниже', 'отметки', 'в', '11', 'тысяч', 'пунктов', ',', 'S', '&', 'P', '500', 'сократился', 'на', '2,63', 'процента', 'до', '1142', 'пунктов', ',', 'а', 'Nasdaq', '-', 'на', '2,76', 'процента', 'до', '2414', 'пунктов.', 'Днем', 'ранее', 'американские', 'индексы', 'выросли', '4', '-', '5', 'процентов.', 'Таким', 'образом', 'инвесторы', 'отреагировали', 'на', 'выступление', 'главы', 'Федеральной', 'резервной', 'системы', 'Бена', 'Бернанке', ',', 'который', 'пообещал', 'сохранить', 'низкие', 'базовые', 'ставки', 'по', 'крайней', 'мере', 'до', '2013', 'года.', 'Кроме', 'того', ',', '9', 'августа', 'американские', 'рынки', 'отыгрывали', 'падение', '8', 'августа', ',', 'когда', 'биржевые', 'показатели', 'сократились', 'на', '5', '-', '6', 'процентов', 'из', '-', 'за'

## Фильтрация по длине токенов

Определяем функцию для фильтрации текстов по количеству токенов. Это необходимо для ограничения длины входных данных в соответствии с возможностями модели.


In [5]:
def select_texts_by_token_count(text_list, pretrained_model, token_limit):
    tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
    result_texts = []
    for sentence in text_list:
        tokenized = tokenizer.tokenize(sentence)
        if len(tokenized) <= token_limit:
            result_texts.append(sentence)
    
    return result_texts

In [6]:
filtered_texts = select_texts_by_token_count(data['text'], "DeepPavlov/rubert-base-cased", 450)

/home/bitcoin/ITMO-DL-NLP/.venv3.10-deeppavlov/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Массовая разметка текстов

Применяем NER-модель ко всем отфильтрованным текстам для получения синтетической разметки. Процесс может занять некоторое время из-за большого объема данных.


In [7]:
synthetic_annots = [ner_model([text]) for text in tqdm(filtered_texts)]
print(synthetic_annots[0])

100%|██████████| 9706/9706 [10:19<00:00, 15.66it/s]

[[['Американские', 'фондовые', 'рынки', 'открылись', '10', 'августа', 'резким', 'снижением', 'котировок.', 'За', 'первые', 'минуты', 'торгов', 'индекс', 'Dow', 'Jones', 'упал', 'на', '2,67', 'процента', 'и', 'снова', 'торгуется', 'ниже', 'отметки', 'в', '11', 'тысяч', 'пунктов', ',', 'S', '&', 'P', '500', 'сократился', 'на', '2,63', 'процента', 'до', '1142', 'пунктов', ',', 'а', 'Nasdaq', '-', 'на', '2,76', 'процента', 'до', '2414', 'пунктов.', 'Днем', 'ранее', 'американские', 'индексы', 'выросли', '4', '-', '5', 'процентов.', 'Таким', 'образом', 'инвесторы', 'отреагировали', 'на', 'выступление', 'главы', 'Федеральной', 'резервной', 'системы', 'Бена', 'Бернанке', ',', 'который', 'пообещал', 'сохранить', 'низкие', 'базовые', 'ставки', 'по', 'крайней', 'мере', 'до', '2013', 'года.', 'Кроме', 'того', ',', '9', 'августа', 'американские', 'рынки', 'отыгрывали', 'падение', '8', 'августа', ',', 'когда', 'биржевые', 'показатели', 'сократились', 'на', '5', '-', '6', 'процентов', 'из', '-', 'за'

## Преобразование схемы разметки

Преобразуем теги из схемы BIES (Begin-Inside-End-Single) в более стандартную схему BIO (Begin-Inside-Outside). Это упрощает дальнейшую работу с данными.

In [9]:
tag_map = {
    'S-LOC': 'B-LOC',
    'E-LOC': 'I-LOC',
    'S-ORG': 'B-ORG',
    'E-ORG': 'I-ORG',
    'S-PER': 'B-PER',
    'E-PER': 'I-PER'
}

adjusted_annots = [[annot[0][0], [tag_map.get(tag, tag) for tag in annot[1][0]]] for annot in synthetic_annots]


## Проверка результатов

Выводим уникальные теги после преобразования для контроля качества разметки и убеждаемся в корректности преобразования схемы.


In [10]:
# Уникальные теги с учётом замен
unique_tags = {tag for annot in adjusted_annots for tag in annot[1]}
print("Уникальные BIO-теги после замены:", unique_tags)

Уникальные BIO-теги после замены: {'O', 'B-LOC', 'I-ORG', 'B-PER', 'I-LOC', 'B-ORG', 'I-PER'}


## Сохранение результатов

Формируем финальный DataFrame с текстами и соответствующими аннотациями, затем сохраняем данные в формате Parquet для дальнейшего использования в задачах машинного обучения.


In [11]:
# Создаем DataFrame с текстами и обновлённой синтетической разметкой
df_annots = pd.DataFrame.from_records(adjusted_annots, columns=['words', 'bio_labels'])

df_annots

,words,bio_labels
0,"[Американские, фондовые, рынки, открылись, 10,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-O..."
1,"[Венесуэла, намерена, ввести, въездные, визы, ...","[B-LOC, O, O, O, O, O, O, B-LOC, O, O, O, O, B..."
2,"[Ученые, из, Австралии, и, Японии, ,, использу...","[O, O, B-LOC, O, B-LOC, O, O, O, O, O, O, O, O..."
3,"[Авианосец, нового, поколения, для, военно, -,...","[O, O, O, O, O, O, O, O, B-LOC, O, O, O, O, O,..."
4,"[Тактика, сеяния, хаоса, ,, которую, ведет, те...","[O, O, O, O, O, O, O, O, O, B-ORG, I-ORG, O, O..."
...,...,...
9701,"[Эстония, вернет, Киеву, конфискованный, на, г...","[B-LOC, O, B-LOC, O, O, O, O, B-LOC, O, O, O, ..."
9702,"[В, 0:00, воскресенья, по, местному, времени, ...","[O, O, O, O, O, O, O, B-LOC, O, O, O, O, O, O,..."
9703,"[Центральная, избирательная, комиссия, Украины...","[B-ORG, I-ORG, I-ORG, B-LOC, O, O, O, O, O, O,..."
9704,"[Интернет, -, компанию, Yahoo, !, разобьют, на...","[O, O, O, B-ORG, B-ORG, O, O, O, O, O, O, O, O..."


In [12]:
# Сохраняем DataFrame в формате Parquet
df_annots.to_parquet('synthetic_annotations.parquet', index=False)